### Script to pull data for specified stations from ACIS, for average temperature and precipitation over a defined time period. Makes plotly scatter plots (which are interactive if desired) and writes them out.

In [1]:
import pandas as pd
import plotly.express as px
import json,requests
from datetime import datetime,time,timedelta

Define some station IDs (as defined in ACIS, so can be ICAO, COOP, etc.)

In [2]:
#ACIS URL
url = 'http://data.rcc-acis.org/StnData'

stn_ids = ["KGJT","053496","053500","053005", "057936","051886","052281","055531",
           "050109","KALS","050848","051121","052220","KPUB","KCOS","058793","055722"]
#stn_ids = ["KGJT"]

Define the time period we're interested in; do some calculations of how many days this is

Note that you can go across a calendar year boundary (for example, from Oct 1 to Sep 30 for a water year), but the code isn't set up to do more than one year. It will also break if you try to go from the middle of one month to earlier in that month (for example, October 15 to October 14 won't work).

In [3]:
## define the period we're interested in
start_month = 7
start_day = 1
end_month = 9
end_day = 30

## set the max number of missing days to allow:
maxmissing = 5

this_year = pd.Timestamp.now().year

start_date = pd.Timestamp(1890,end_month,end_day) # this needs to have the *end* of the period, but some early year
end_date = pd.Timestamp(this_year,end_month,end_day)

## how many days are in our period of interest?
## account for the end month being earlier than the start month, so going over the year switch
if end_month >= start_month:
    numdays = (pd.Timestamp(this_year,end_month,end_day) - pd.Timestamp(this_year,start_month,start_day)).days
else:
    numdays = (pd.Timestamp(this_year,end_month,end_day) - pd.Timestamp(this_year-1,start_month,start_day)).days

first_day_text = pd.Timestamp(this_year,start_month,start_day).strftime("%B %-d")
last_day_text = pd.Timestamp(this_year,end_month,end_day).strftime("%B %-d")
print("averaging over "+str(numdays)+" days from "+first_day_text+" to "+last_day_text) 

first_day_abbrev=pd.Timestamp(this_year,start_month,start_day).strftime("%d%b")
last_day_abbrev=pd.Timestamp(this_year,end_month,end_day).strftime("%d%b")

averaging over 91 days from July 1 to September 30


Now loop over the stations, pull the data, and make the plots! You shouldn't need to change this code at all, unless you want to alter how the plots look, etc.

In [10]:
for stn_id in stn_ids:
    
    ## get data from ACIS
    print("end date: "+end_date.strftime('%Y-%m-%d'))   
    print("working on station "+stn_id)

    ## this will get the mean over a numdays-long period, one value per year ending on the specified month/day
    print("avg temp")

    try:
        payload = {
        "output": "json",
        "params": {"elems":[{"name":"avgt","interval":[1,0,0],"duration":numdays,
                             "reduce":"mean","maxmissing":maxmissing}],
                   "sid":stn_id,"sdate":start_date.strftime('%Y-%m-%d'),"edate":end_date.strftime('%Y-%m-%d')} 
        }


        r = requests.post(url, json=payload)
        data = r.json()

        stn_name = data['meta']['name']
        print("this station is: "+stn_name)

        data_pd = pd.DataFrame(data['data'], columns=['date','tavg'])
        data_pd['tavg'] = pd.to_numeric(data_pd['tavg'], errors='coerce')

        ## repeat for precip
        ## this will get the mean over a numdays-long period, one value per year ending on the specified month/day
        print("precip")
        payload = {
        "output": "json",
        "params": {"elems":[{"name":"pcpn","interval":[1,0,0],"duration":numdays,
                             "reduce":"sum","maxmissing":maxmissing}],
                   "sid":stn_id,"sdate":start_date.strftime('%Y-%m-%d'),"edate":end_date.strftime('%Y-%m-%d')} 
        }


        r = requests.post(url, json=payload)
        data = r.json()

        data_pd2 = pd.DataFrame(data['data'], columns=['date','prcp'])
        data_pd2['prcp'] = pd.to_numeric(data_pd2['prcp'], errors='coerce')

        ### also get the normals for this time period
        print("temp normal")
        payload = {
        "output": "json",
        "params": {"elems":[{"name":"avgt","interval":[1,0,0],"duration":numdays,
                             "reduce":"mean","maxmissing":maxmissing,"normal":"1"}],
                   "sid":stn_id,"date":end_date.strftime('%Y-%m-%d')} 
        }

        r = requests.post(url, json=payload)
        data = r.json()

        data_pd3 = pd.DataFrame(data['data'], columns=['date','tavg_normal'])
        data_pd3['tavg_normal'] = pd.to_numeric(data_pd3['tavg_normal'], errors='coerce')

        print("precip normal")
        payload = {
        "output": "json",
        "params": {"elems":[{"name":"pcpn","interval":[1,0,0],"duration":numdays,
                             "reduce":"sum","maxmissing":maxmissing,"normal":"1"}],
                   "sid":stn_id,"date":end_date.strftime('%Y-%m-%d')} 
        }

        r = requests.post(url, json=payload)
        data = r.json()

        data_pd4 = pd.DataFrame(data['data'], columns=['date','prcp_normal'])
        data_pd4['prcp_normal'] = pd.to_numeric(data_pd4['prcp_normal'], errors='coerce')

        ## and merge the data frames and set the index
        data = data_pd.merge(data_pd2, on='date').set_index(pd.to_datetime(data_pd['date']))

        ## make a year column for plotting
        data['year'] = data.index.year

        ## and make the plot with plotly
        fig = px.scatter(data.dropna(), x="prcp", y="tavg",
                    hover_data=['prcp','tavg','year'],
                    labels = {'tavg':'avg temp (F)','prcp':'accumulated precipitation (inches)'},
                    size='prcp', color='tavg',
                     color_continuous_scale=px.colors.sequential.matter,
                    text='year',
                    width=850,height=750)

        fig.update_traces(textposition='top center',textfont_size=9)

        # add lines for the normals
        fig.add_vline(data_pd4.prcp_normal[0], line_dash='dash',line_color='gray',opacity=0.6)
        fig.add_hline(data_pd3.tavg_normal[0], line_dash='dash',line_color='gray',opacity=0.6)

        fig.update_layout(
            title={
                'text': stn_name+' temperature and precipitation, '+first_day_text+' - '+last_day_text,
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                           showlegend=True,
                              annotations=[
                                  dict(x=1.125,y=-0.1,showarrow=False,
                                       text="Data source: ACIS",
                                       xref="paper",yref="paper",font={'size':10.5}),
                                  dict(x=-0.075,y=-0.1,showarrow=False,
                                       text="size of points proportional to precip,<br>color shows temp<br>normals are 1981-2010",
                                       xref="paper",yref="paper",font={'size':10.5}),
                                  dict(x=0,y=1,showarrow=False,
                                       text="warm & dry",
                                       xref="paper",yref="paper",font={'size':14, 'color':'brown'}),
                                  dict(x=1,y=0,showarrow=False,
                                       text="cool & wet",
                                       xref="paper",yref="paper",font={'size':14, 'color':'blue'}),
                                  dict(x=1,y=1,showarrow=False,
                                       text="warm & wet",
                                       xref="paper",yref="paper",font={'size':14, 'color':'brown'}),
                                  dict(x=0,y=0,showarrow=False,
                                       text="cool & dry",
                                       xref="paper",yref="paper",font={'size':14, 'color':'blue'}),
                                  dict(x=data_pd4.prcp_normal[0]+0.9,y=0,showarrow=False,
                                       text="normal precip: "+str(data_pd4.prcp_normal[0])+"\"",
                                       yref="paper",font={'size':11, 'color':'gray'}), 
                                  dict(x=0,y=data_pd3.tavg_normal[0]+0.15,showarrow=False,
                                       text="normal TAVG: "+str(data_pd3.tavg_normal[0])+"F",
                                       xref="paper",font={'size':11, 'color':'gray'}),   
                                          ])
        
        ## this code controls how the png gets written if you click the 'save' button
        ## in the interactive graph
        config = {
        'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': "tavg_prcp_scatter_"+stn_id+"_"+first_day_abbrev+"_"+last_day_abbrev,
        'width': 850,
        'height': 750,
        'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
         }
         }

        fig.write_image("tavg_prcp_scatter_"+stn_id+"_"+first_day_abbrev+"_"+last_day_abbrev+".pdf", 
                        width=850, height=750, engine='kaleido')

        ## show interactive graph:
        fig.show(config=config)
        
    except:
        print("something went wrong for this station, going on to the next one")



end date: 2020-09-30
working on station KGJT
avg temp
this station is: GRAND JUNCTION WALKER FIELD
precip
temp normal
precip normal
